In [1]:
# load the functionality into Julia
using StatsFuns, CSV, Cubature, ProgressMeter, DataFrames, Optim, SpecialFunctions, Roots
using Distributions, DelimitedFiles, Random

using Plots, StatsPlots, ColorBrewer
pyplot()

Plots.PyPlotBackend()

In [2]:
Plots.scalefontsizes(1.2)

In [3]:
clrs = [ColorBrewer.palette("Blues", 5)[2], ColorBrewer.palette("Blues", 5)[4]];
clrs

In [4]:
# output file
results_dir = "../../results/Andrei"
results_file = "$(results_dir)/output.csv"
run(`mkdir -p $(results_dir)`)

Process(`mkdir -p ../../results/Andrei`, ProcessExited(0))

In [5]:
header = ["filename","distribution","truncated","mean","sd","negloglk","AIC","mean_lower","mean_upper",
    "sd_lower","sd_upper",
    "q5_lower","q5_median","q5_upper",
    "q50_lower","q50_median","q50_upper",
    "q95_lower","q95_median","q95_upper",
    "q99_lower","q99_median","q99_upper"]
header=reshape(header,1,length(header))
open(results_file, "w") do io
    writedlm(io, header, ',')
end

In [6]:
## The function is adopted from:
## from https://github.com/robertfeldt/BlackBoxOptim.jl/blob/master/src/utilities/latin_hypercube_sampling.jl
"""
    latin_hypercube_sampling(mins, maxs, numSamples)
    Randomly sample `numSamples` values from the parallelogram defined
    by `mins` and `maxs` using the Latin hypercube algorithm.
"""
function latin_hypercube_sampling(mins::AbstractVector{T},
                                  maxs::AbstractVector{T},
                                  n::Integer) where T<:Number
    length(mins) == length(maxs) ||
        throw(DimensionMismatch("mins and maxs should have the same length"))
    all(xy -> xy[1] <= xy[2], zip(mins, maxs)) ||
        throw(ArgumentError("mins[i] should not exceed maxs[i]"))
    dims = length(mins)
    result = zeros(T, dims, n)
    cubedim = Vector{T}(undef, n)
    @inbounds for i in 1:dims
        imin = mins[i]
        dimstep = (maxs[i] - imin) / n
        for j in 1:n
            cubedim[j] = imin + dimstep * (j - 1 + rand(T))
        end
        result[i, :] .= shuffle!(cubedim)
    end
    return result
end

nsamples = 10000
# for reproducibility we fix the seed of the random number generator
smpls = latin_hypercube_sampling([0.0],[1.0],nsamples);

In [7]:
filenames = ["data_incper", "data_incper_inclwuhan", "data_ons_hosp", 
    "dthdata_hosp_dth", "dthdata_ons_dth", "dthdata_ons_hosp"]

6-element Array{String,1}:
 "data_incper"          
 "data_incper_inclwuhan"
 "data_ons_hosp"        
 "dthdata_hosp_dth"     
 "dthdata_ons_dth"      
 "dthdata_ons_hosp"     

In [8]:
df = DataFrame(CSV.read("../../data/$(filenames[2]).csv", missingstring="NA"))
select!(df, [:EL, :ER, :SL, :SR, :tstar])
rename!(df, Dict(:EL=>:E_L, :ER=>:E_R, :SL=>:S_L, :SR=>:S_R))

,E_L,E_R,S_L,S_R,tstar
,Int64,Int64,Int64,Int64,Int64
1,28,33,33,34,62
2,0,35,35,36,62
3,0,43,43,44,62
4,0,48,48,49,62
5,37,43,43,44,62
6,39,44,44,45,62
7,0,43,43,44,62
8,0,42,45,46,62
9,38,41,41,47,62


In [ ]:
flname = filenames[1]
idx=1
println(flname)
df = DataFrame(CSV.read("../../data/$(flname).csv", missingstring="NA"))
select!(df, [:EL, :ER, :SL, :SR, :tstar])
rename!(df, Dict(:EL=>:E_L, :ER=>:E_R, :SL=>:S_L, :SR=>:S_R))

logmeans_inits = log.([4.2, 5.5, 3.7, 5., 6.8, 7.2])
logsd_inits = log.([3.2, 5.5, 6.2, 5., 5.8, 5.5])

if (idx==2)
    df[df[!,:E_L].==0,:E_L] .= df[df[!,:E_L].==0,:S_L] .- 28.
end

prog = ProgressThresh(-Inf, "Without truncation:") 
function getNegativeLoglk(params)
    mean_SI = exp(params[1]) # make the parameters positive
    sd_SI = exp(params[2]) # make the parameters positive
    shape = (mean_SI/sd_SI)^2
    invscale = (sd_SI^2)/mean_SI
    loglk = 0
    @inline function Fs(t) 
        gammacdf(shape, invscale, t)
    end
    # this for-loop is based on coarseData R package consolidated to one piece of code
    @simd for k in 1:nrow(df)
        E_R = df[k,:E_R]; E_L = df[k,:E_L]; S_R = df[k,:S_R]; S_L = df[k,:S_L]
        if (S_L!=S_R)
            if (S_L > E_R) # if the intervals do not overlap
                loglk += log(hquadrature(e -> (Fs(S_R-e) - Fs(S_L-e)), E_L, E_R)[1])
            elseif (S_L > E_L)
                loglk += log(hquadrature(e -> (Fs(S_R-e) - Fs(S_L-e)), E_L, S_L)[1] + hquadrature(e -> Fs(S_R-e), S_L, E_R)[1])
            else
                loglk += log(hquadrature(e -> Fs(S_R-e), E_L, E_R)[1])
            end
        else 
            println("There are S_L = S_R!")
        end
    end
    ProgressMeter.update!(prog, -loglk)
    return(-loglk)
end

println("* MLE")
res0 = optimize(x -> getNegativeLoglk(x), [logmeans_inits[idx], logsd_inits[idx]])

println("* Generating the results")
logprms = Optim.minimizer(res0)
prms = exp.(logprms)
println([(prms[1]/prms[2])^2, prms[2]^2/prms[1]])
logmeans_inits[idx] = logprms[1]
logsd_inits[idx] = logprms[2]

negloglk = Optim.minimum(res0)
AIC = -2*2 + 2negloglk

res = [prms[1], prms[2], negloglk, AIC]

println("* profile-based for the mean")
step = .16
function y(s) 
    y = Optim.minimum(optimize(x -> getNegativeLoglk([s x]), [logsd_inits[idx]], LBFGS()))
    println(y)
    return  y - negloglk - 1.92
end

# x0 = logprms[1]
# y0s = -1.92
# while y0s<0
#     x0 -= step
#     y0s = y(x0)
#     println(y0s)
# end
logprms1_lower = fzero(y, logsd_inits[idx])[1]


# logprms1_lower = fzero(y, logprms[2]-0.01)[1]

Without truncation: (thresh = -Inf, value = Inf)

In [28]:
logprms1_lower = fzero(y, x0)[1]

Without truncation: (thresh = -Inf, value = 58.2234)

InterruptException: InterruptException:

In [ ]:
func = TwiceDifferentiable(vars -> Log_Likelihood(x, y, vars[1:nvar], vars[nvar + 1]),
                           ones(nvar+1); autodiff=:forward);

In [ ]:
logmeans_inits = log.([4.2, 5.5, 3.7, 5., 6.8, 7.2])
logsd_inits = log.([3.2, 5.5, 6.2, 5., 5.8, 5.5])

In [64]:
logprms[2]

1.0900026440345811

In [8]:
?expm1

search:

InterruptException: InterruptException:

# Without truncation

In [39]:
filenames = ["data_incper", "data_incper_inclwuhan", "data_ons_hosp", 
    "dthdata_hosp_dth", "dthdata_ons_dth", "dthdata_ons_hosp"]
tmax = [16, 16, 16, 35, 35, 35]
tmstps = [4, 4, 4, 7, 7, 7]

logmeans_inits = log.([4.2, 5.5, 3.7, 5., 6.8, 7.2])
logsd_inits = log.([3.2, 5.5, 6.2, 5., 5.8, 5.5])

idx = 1
for flname in [filenames[1]]#[idx:end]
    println(flname)
    df = DataFrame(CSV.read("../../data/$(flname).csv", missingstring="NA"))
    select!(df, [:EL, :ER, :SL, :SR, :tstar])
    rename!(df, Dict(:EL=>:E_L, :ER=>:E_R, :SL=>:S_L, :SR=>:S_R))
    
    if (idx==2)
        df[df[!,:E_L].==0,:E_L] .= df[df[!,:E_L].==0,:S_L] .- 28.
    end
    
    prog = ProgressThresh(-Inf, "Without truncation:") 
    function getNegativeLoglk(params)
        mean_SI = exp(params[1]) # make the parameters positive
        sd_SI = exp(params[2]) # make the parameters positive
        shape = (mean_SI/sd_SI)^2
        invscale = (sd_SI^2)/mean_SI
        loglk = 0
        @inline function Fs(t) 
            gammacdf(shape, invscale, t)
        end
        # this for-loop is based on coarseData R package consolidated to one piece of code
        @simd for k in 1:nrow(df)
            E_R = df[k,:E_R]; E_L = df[k,:E_L]; S_R = df[k,:S_R]; S_L = df[k,:S_L]
            if (S_L!=S_R)
                if (S_L > E_R) # if the intervals do not overlap
                    loglk += log(hquadrature(e -> (Fs(S_R-e) - Fs(S_L-e)), E_L, E_R)[1])
                elseif (S_L > E_L)
                    loglk += log(hquadrature(e -> (Fs(S_R-e) - Fs(S_L-e)), E_L, S_L)[1] + hquadrature(e -> Fs(S_R-e), S_L, E_R)[1])
                else
                    loglk += log(hquadrature(e -> Fs(S_R-e), E_L, E_R)[1])
                end
            else 
                println("There are S_L = S_R!")
            end
        end
        ProgressMeter.update!(prog, -loglk)
        return(-loglk)
    end
    
    println("* MLE")
    res0 = optimize(x -> getNegativeLoglk(x), [logmeans_inits[idx], logsd_inits[idx]])
    
    println("* Generating the results")
    logprms = Optim.minimizer(res0)
    prms = exp.(logprms)
    println([(prms[1]/prms[2])^2, prms[2]^2/prms[1]])
    logmeans_inits[idx] = logprms[1]
    logsd_inits[idx] = logprms[2]

    negloglk = Optim.minimum(res0)
    AIC = -2*2 + 2negloglk

    res = [prms[1], prms[2], negloglk, AIC]
    
    println("* profile-based for the mean")
    step = .1
    function y(logs) 
        y = Optim.minimum(optimize(x -> getNegativeLoglk([logs, x]), logprms[2]))
        return  y - negloglk - 1.92
    end
    x0 = logprms[1]
    while y(x0)<0
        x0 -= step
    end
    logprms1_lower = fzero(y, x0)[1]

    x0 = logprms[1]
    while y(x0)<0
        x0 += step
    end
    logprms1_upper = fzero(y, (logprms[1], x0))[1]

    res_ = exp.([logprms1_lower, logprms1_upper])
    res = vcat(res, res_)
    means_samples = latin_hypercube_sampling([res_[1]],[res_[2]],nsamples);
    
    println("* profile-based for the SD")
    step = .1
    y = x -> getNegativeLoglk([logprms[1], x]) - negloglk - 1.92
    x0 = logprms[2]
    while y(x0)<0
        x0 -= step
    end
    logprms1_lower = fzero(y, (x0,logprms[2]))[1]

    x0 = logprms[2]
    while y(x0)<0
        x0 += step
    end
    logprms1_upper = fzero(y, (logprms[2], x0))[1]

    res_ = exp.([logprms1_lower, logprms1_upper])
    res = vcat(res, res_)
    SD_samples = latin_hypercube_sampling([res_[1]],[res_[2]],nsamples);
    
    println("* Creating a plot")
    t = [0:0.05:tmax[idx];]
    for idx in 1:nsamples
        shape = (means_samples[idx]/SD_samples[idx])^2
        scale = SD_samples[idx]^2/means_samples[idx]
        if idx>1
            append!(y,[cdf.(Gamma(shape, scale),t)])
        else 
            y = [cdf.(Gamma(shape, scale),t)]
        end
    end
    plot(t, y[1:100], legend=false, color=clrs[1], alpha=.3, 
        grid=false, yticks = 0:0.2:1, xticks = 0:tmstps[idx]:tmax[idx],
        xlabel="Exposure to onset (days)", ylabel="Proportion with symptoms",
        size = (300,400), dpi = 300)
    plot!(t, median(hcat(y...);dims=2), color=clrs[2], lw=1.4)

    quantiles = [0.05,.5,.95,.99]
    for idx in 1:nsamples
        shape = (means_samples[idx]/SD_samples[idx])^2
        scale = SD_samples[idx]^2/means_samples[idx]
        if idx>1
            append!(y,[quantile.(Gamma(shape, scale),quantiles)])
        else 
            y = [quantile.(Gamma(shape, scale),quantiles)]
        end
    end
    y = hcat(y...)
    for i in 1:length(quantiles)
        if i<=3
            plot!(quantile(y[i,:],[0.025,.975]),[quantiles[i],quantiles[i]], color=clrs[2])
        end
    end
    scatter!(median(y;dims=2)[1:(end-1)], quantiles[1:(end-1)], 
        color=clrs[2], markercolor=clrs[2], markersize=5) 

    Plots.pdf("$(results_dir)/gamma-$(flname)-no_truncation.pdf")
    
    for i in 1:length(quantiles)
        res_ = quantile(y[i,:],[0.025, .5, .975])
        res = vcat(res, res_)
    end
    
    println(res)
    
    res=reshape(vcat(["$(flname)","Gamma", "no"],round.(res,digits=4)),1,length(res)+3)
    open(results_file, "a") do io
        writedlm(io, res, ',')
    end
    
    idx += 1
end

data_incper
* MLE


Without truncation: (thresh = -Inf, value = 57.4362)

* Generating the results
[2.955527476763615, 1.7300437793494186]
* profile-based for the mean


MethodError: MethodError: no method matching optimize(::var"#151#162"{Float64,var"#getNegativeLoglk#154"{ProgressThresh{Float64}}}, ::Float64)
Closest candidates are:
  optimize(::F, ::T, !Matched::T; method, rel_tol, abs_tol, iterations, store_trace, show_trace, callback, show_every, extended_trace) where {F<:Function, T<:AbstractFloat} at /home/aakhmetz/.julia/packages/Optim/SFpsz/src/univariate/optimize/interface.jl:14
  optimize(::F, ::T, !Matched::T, !Matched::GoldenSection; rel_tol, abs_tol, iterations, store_trace, show_trace, callback, show_every, extended_trace, nargs...) where {F<:Function, T<:AbstractFloat} at /home/aakhmetz/.julia/packages/Optim/SFpsz/src/univariate/solvers/golden_section.jl:54
  optimize(::F, ::T, !Matched::T, !Matched::Brent; rel_tol, abs_tol, iterations, store_trace, show_trace, callback, show_every, extended_trace) where {F<:Function, T<:AbstractFloat} at /home/aakhmetz/.julia/packages/Optim/SFpsz/src/univariate/solvers/brent.jl:58
  ...

In [35]:
res0

UndefVarError: UndefVarError: res0 not defined

# With truncation

In [34]:
filenames = ["data_incper", "data_incper_inclwuhan", "data_ons_hosp", 
    "dthdata_hosp_dth", "dthdata_ons_dth", "dthdata_ons_hosp"]
tmax = [16, 16, 16, 35, 35, 16]
tmstps = [4, 4, 4, 7, 7, 7]

logmeans_inits = log.([4.2, 4.2, 3.7, 5., 6.2, 7.2])
logsd_inits = log.([3.1, 6.05, 6.2, 5., 5.8, 5.5])

idx = 2
for flname in filenames[idx:end]
    println(flname)
    df = DataFrame(CSV.read("../../data/$(flname).csv", missingstring="NA"))
    select!(df, [:EL, :ER, :SL, :SR, :tstar])
    rename!(df, Dict(:EL=>:E_L, :ER=>:E_R, :SL=>:S_L, :SR=>:S_R))
    
    if (idx==2)
        df[df[!,:E_L].==0,:E_L] .= df[df[!,:E_L].==0,:S_L] .- 28.
    end
    
    prog = ProgressThresh(-Inf, "With truncation:") 
    function getNegativeLoglk(params)
        r = 0.14
        mean_SI = exp(params[1]) # make the parameters positive
        sd_SI = exp(params[2]) # make the parameters positive
        shape = (mean_SI/sd_SI)^2
        invscale = (sd_SI^2)/mean_SI
        loglk = 0
        @inline function fs(t) 
            gammapdf(shape, invscale, t)
        end
        @inline function Fs(t) 
            gammacdf(shape, invscale, t)
        end
        # this for-loop is based on coarseData R package consolidated to one piece of code
        @simd for k in 1:nrow(df)
            E_R = df[k,:E_R]; E_L = df[k,:E_L]; S_R = df[k,:S_R]; S_L = df[k,:S_L]; tstar = df[k,:tstar]
            @inline function g(e)
                hquadrature(x -> r*exp(-r*x)/(1-exp(-r*x))*Fs(tstar-e-x), 0.001, tstar-e)[1]
            end
            if (S_L!=S_R)
                if (S_L > E_R) # if the intervals do not overlap
                    loglk += log(hquadrature(e -> (Fs(S_R-e) - Fs(S_L-e))/g(e), E_L, E_R)[1])
                elseif (S_L > E_L)
                    loglk += log(hquadrature(e -> (Fs(S_R-e) - Fs(S_L-e))/g(e), E_L, S_L)[1] + 
                                 hquadrature(e -> Fs(S_R-e)/g(e), S_L, E_R)[1])
                else
                    loglk += log(hquadrature(e -> Fs(S_R-e)/g(e), E_L, E_R)[1])
                end
            else
                println("There are some S_L = S_R!")
            end
        end
        ProgressMeter.update!(prog, -loglk)
        return(-loglk)
    end
    
    println("* MLE")
    res0 = optimize(x -> getNegativeLoglk(x), [logmeans_inits[idx], logsd_inits[idx]])
    
    println("* Generating the results")
    logprms = Optim.minimizer(res0)
    prms = exp.(logprms)
    logmeans_inits[idx] = logprms[1]
    logsd_inits[idx] = logprms[2]

    negloglk = Optim.minimum(res0)
    AIC = -2*2 + 2negloglk

    res = [prms[1], prms[2], negloglk, AIC]
    
    println("* profile-based for the mean")
    step = .1
    y = x -> getNegativeLoglk([x, logprms[2]]) - negloglk - 1.92
    x0 = logprms[1]
    while y(x0)<0
        x0 -= step
    end
    logprms1_lower = fzero(y, (x0,logprms[1]))[1]

    x0 = logprms[1]
    while y(x0)<0
        x0 += step
    end
    logprms1_upper = fzero(y, (logprms[1], x0))[1]

    res_ = exp.([logprms1_lower, logprms1_upper])
    res = vcat(res, res_)
    means_samples = latin_hypercube_sampling([res_[1]],[res_[2]],nsamples);
    
    println("* profile-based for the SD")
    step = .1
    y = x -> getNegativeLoglk([logprms[1], x]) - negloglk - 1.92
    x0 = logprms[2]
    while y(x0)<0
        x0 -= step
    end
    logprms1_lower = fzero(y, (x0,logprms[2]))[1]

    x0 = logprms[2]
    while y(x0)<0
        x0 += step
    end
    logprms1_upper = fzero(y, (logprms[2], x0))[1]

    res_ = exp.([logprms1_lower, logprms1_upper])
    res = vcat(res, res_)
    SD_samples = latin_hypercube_sampling([res_[1]],[res_[2]],nsamples);
    
    println("* Creating a plot")
    t = [0:0.05:tmax[idx];]
    for idx in 1:nsamples
        shape = (means_samples[idx]/SD_samples[idx])^2
        scale = SD_samples[idx]^2/means_samples[idx]
        if idx>1
            append!(y,[cdf.(Gamma(shape, scale),t)])
        else 
            y = [cdf.(Gamma(shape, scale),t)]
        end
    end
    plot(t, y[1:100], legend=false, color=clrs[1], alpha=.3, 
        grid=false, yticks = 0:0.2:1, xticks = 0:tmstps[idx]:tmax[idx],
        xlabel="Exposure to onset (days)", ylabel="Proportion with symptoms",
        size = (300,400), dpi = 300)
    plot!(t, median(hcat(y...);dims=2), color=clrs[2], lw=1.4)

    quantiles = [0.05,.5,.95,.99]
    for idx in 1:nsamples
        shape = (means_samples[idx]/SD_samples[idx])^2
        scale = SD_samples[idx]^2/means_samples[idx]
        if idx>1
            append!(y,[quantile.(Gamma(shape, scale),quantiles)])
        else 
            y = [quantile.(Gamma(shape, scale),quantiles)]
        end
    end
    y = hcat(y...)
    for i in 1:length(quantiles)
        if i<=3
            plot!(quantile(y[i,:],[0.025,.975]),[quantiles[i],quantiles[i]], color=clrs[2])
        end
    end
    scatter!(median(y;dims=2)[1:(end-1)], quantiles[1:(end-1)], 
        color=clrs[2], markercolor=clrs[2], markersize=5) 

    Plots.pdf("$(results_dir)/gamma-$(flname)-with_truncation.pdf")
    
    for i in 1:length(quantiles)
        res_ = quantile(y[i,:],[0.025, .5, .975])
        res = vcat(res, res_)
    end
    
    println(res)
    
    res=reshape(vcat(["$(flname)","Gamma", "yes"],round.(res,digits=4)),1,length(res)+3)
    open(results_file, "a") do io
        writedlm(io, res, ',')
    end
    
    idx += 1
end

data_incper_inclwuhan
* MLE


With truncation: (thresh = -Inf, value = 379.296)

* Generating the results
* profile-based for the mean


With truncation: (thresh = -Inf, value = 381.216)

* profile-based for the SD


With truncation: (thresh = -Inf, value = 381.216)

InterruptException: InterruptException: